In [1]:
import re
from pathlib import Path
from pprint import pp

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import (
    PredictionErrorDisplay,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from P08_feature_importances.T00_lib.classes_ml import MyUtil
from P08_feature_importances.T00_lib.utils import check_jupyter

In [2]:
SAVE_DATA = False
MAKE_ONE_HOT = False

In [3]:
if check_jupyter():
    BASE_DIR = Path.cwd()  # Current directory of the running file
    DATA_DIR = BASE_DIR.parent / "T02_combine_features"
    CURRENT_DIR = BASE_DIR

dt = MyUtil.get_dt()
print(f"Current Directory: {CURRENT_DIR}")
print(f"Current Date and Time: {dt}")

# Load data
df = pd.read_excel(DATA_DIR / "S02_data_combined_loc.xlsx")
print(df.shape)
df

Code is running in a Jupyter environment.
Current Directory: c:\Users\admin\Coding\research\weld-ml\src\P08_feature_importances\T11_tabPFN
Current Date and Time: 2026-01-27_13-51
(378, 87)


,sample_no,R,W,D,position,location,Fx_location,Fy_location,Fz_location,Mz_location,...,"Mz__weld__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2","Mz__weld__fft_coefficient__attr_""angle""__coeff_15",Mz__weld__last_location_of_maximum,Mz__weld__first_location_of_maximum,"Mz__weld__fft_coefficient__attr_""abs""__coeff_58","Mz__weld__fft_coefficient__attr_""abs""__coeff_83","Mz__weld__fft_coefficient__attr_""abs""__coeff_96",stress_value_5052,stress_value_6061,stress_value_center
0,1,1400,60,10,0.153846,1,-0.077671,0.143026,1.244326,1.873865,...,-0.197924,93.404989,0.034228,0.034151,770.788243,599.048302,791.064441,28.0,51.0,12.0
1,2,1400,60,15,0.153846,1,-0.133276,0.164254,1.203367,-1.054677,...,-0.105672,40.096726,0.010845,0.010768,358.604392,111.493021,204.891839,14.0,-21.0,17.0
2,3,1400,60,20,0.153846,1,-0.059639,0.269418,1.444542,2.940728,...,-0.088083,149.473672,0.046766,0.046689,292.417343,171.931801,215.611943,10.0,35.0,12.0
3,4,1400,70,10,0.153846,1,-0.051020,0.211907,1.601667,3.661974,...,-0.033330,75.561565,0.055815,0.055725,311.972632,198.723230,176.177248,10.0,-10.0,20.0
4,5,1400,70,15,0.153846,1,-0.100744,0.179582,1.095031,-0.016799,...,-0.220260,85.348076,0.000179,0.000090,601.144018,368.952328,557.228141,6.0,41.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,50,1600,70,15,0.846154,7,-0.084714,0.284958,1.905742,8.950274,...,-0.306988,87.390014,0.989411,0.989322,823.146921,429.867587,1175.677841,4.0,-23.0,2.0
374,51,1600,70,20,0.846154,7,-0.094956,0.257101,1.669120,9.627879,...,-0.201139,65.122289,0.975592,0.975503,979.362101,773.237350,771.845687,0.0,-1.0,2.0
375,52,1600,80,10,0.846154,7,-0.203323,0.173404,1.671576,4.696642,...,-0.299690,58.113275,0.005230,0.005128,663.567303,623.167416,430.622244,-2.0,-41.0,5.0
376,53,1600,80,15,0.846154,7,-0.099644,0.266207,1.686495,10.497974,...,-0.072368,68.126310,0.934468,0.934366,499.039605,280.783456,645.597969,10.0,-90.0,1.0


In [4]:
colsY = [c for c in df.columns if re.search(r"stress_value", c)]
print(colsY)
colsX = [c for c in df.columns if c not in colsY]
print(colsX)

['stress_value_5052', 'stress_value_6061', 'stress_value_center']
['sample_no', 'R', 'W', 'D', 'position', 'location', 'Fx_location', 'Fy_location', 'Fz_location', 'Mz_location', 'loc_idx', 'loc_time', 'Fx__dwell__fft_coefficient__attr_"abs"__coeff_11', 'Fx__dwell__ar_coefficient__coeff_0__k_10', 'Fx__dwell__variation_coefficient', 'Fx__dwell__quantile__q_0.7', 'Fx__dwell__fft_coefficient__attr_"real"__coeff_62', 'Fx__dwell__fft_coefficient__attr_"real"__coeff_42', 'Fx__dwell__partial_autocorrelation__lag_6', 'Fx__dwell__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'Fx__dwell__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2', 'Fy__dwell__fft_coefficient__attr_"real"__coeff_71', 'Fy__dwell__fft_coefficient__attr_"abs"__coeff_59', 'Fy__dwell__fft_coefficient__attr_"abs"__coeff_46', 'Fy__dwell__last_location_of_minimum', 'Fy__dwell__first_location_of_minimum', 'Fy__dwell__fft_coefficient__attr_"real"__coeff_16', 'Fy__dwell__change_quantiles__f_agg_"mean"__isabs_

In [5]:
dfY = df[colsY]
dfY = dfY[["stress_value_center"]]
dfX_all = df[colsX]

print(dfX_all.shape)
print(dfY.shape)

(378, 84)
(378, 1)


In [6]:
if MAKE_ONE_HOT:
    dft1 = pd.get_dummies(
        dfX_all, columns=["location", "R", "W", "D"], drop_first=False, dtype=int
    )
    dfX = dft1.drop(columns=["sample_no"])
    colsX = dfX.columns.tolist()
else:
    dfX = dfX_all.drop(columns=["sample_no", "location"])
    colsX = dfX.columns.tolist()
dfX

,R,W,D,position,Fx_location,Fy_location,Fz_location,Mz_location,loc_idx,loc_time,...,Fz__weld__ratio_beyond_r_sigma__r_2.5,Mz__weld__quantile__q_0.1,Mz__weld__energy_ratio_by_chunks__num_segments_10__segment_focus_4,"Mz__weld__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.2","Mz__weld__fft_coefficient__attr_""angle""__coeff_15",Mz__weld__last_location_of_maximum,Mz__weld__first_location_of_maximum,"Mz__weld__fft_coefficient__attr_""abs""__coeff_58","Mz__weld__fft_coefficient__attr_""abs""__coeff_83","Mz__weld__fft_coefficient__attr_""abs""__coeff_96"
0,1400,60,10,0.153846,-0.077671,0.143026,1.244326,1.873865,5856,58.560000,...,0.011999,-6.949290,0.034026,-0.197924,93.404989,0.034228,0.034151,770.788243,599.048302,791.064441
1,1400,60,15,0.153846,-0.133276,0.164254,1.203367,-1.054677,6262,62.620000,...,0.026536,-5.712890,0.021547,-0.105672,40.096726,0.010845,0.010768,358.604392,111.493021,204.891839
2,1400,60,20,0.153846,-0.059639,0.269418,1.444542,2.940728,6824,68.240000,...,0.046766,-5.951800,0.024692,-0.088083,149.473672,0.046766,0.046689,292.417343,171.931801,215.611943
3,1400,70,10,0.153846,-0.051020,0.211907,1.601667,3.661974,5378,53.782857,...,0.032843,-4.033597,0.032070,-0.033330,75.561565,0.055815,0.055725,311.972632,198.723230,176.177248
4,1400,70,15,0.153846,-0.100744,0.179582,1.095031,-0.016799,5959,59.592857,...,0.017408,-7.551309,0.018795,-0.220260,85.348076,0.000179,0.000090,601.144018,368.952328,557.228141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,1600,70,15,0.846154,-0.084714,0.284958,1.905742,8.950274,13757,137.565714,...,0.012383,-6.994062,0.024102,-0.306988,87.390014,0.989411,0.989322,823.146921,429.867587,1175.677841
374,1600,70,20,0.846154,-0.094956,0.257101,1.669120,9.627879,14329,143.285714,...,0.000090,-8.050406,0.019972,-0.201139,65.122289,0.975592,0.975503,979.362101,773.237350,771.845687
375,1600,80,10,0.846154,-0.203323,0.173404,1.671576,4.696642,12049,120.490000,...,0.011999,-18.062500,0.024198,-0.299690,58.113275,0.005230,0.005128,663.567303,623.167416,430.622244
376,1600,80,15,0.846154,-0.099644,0.266207,1.686495,10.497974,12631,126.310000,...,0.042150,-4.259820,0.027400,-0.072368,68.126310,0.934468,0.934366,499.039605,280.783456,645.597969


In [7]:
# Extract features and targets
_X = dfX.values
_Y = dfY.values

print(_X.shape)
print(_Y.shape)

(378, 82)
(378, 1)


In [8]:
_X_train, _X_test, _Y_train, _Y_test = train_test_split(
    _X, _Y, test_size=0.3, random_state=1
)
print(_X_train.shape)
print(_X_test.shape)
print(_Y_train.shape)
print(_Y_test.shape)

(264, 82)
(114, 82)
(264, 1)
(114, 1)


In [9]:
scX = StandardScaler()
X_train = scX.fit_transform(_X_train)
X_test = scX.transform(_X_test)

scY = StandardScaler()
Y_train = scY.fit_transform(_Y_train)
Y_test = scY.transform(_Y_test)

X_train_df = pd.DataFrame(X_train, columns=colsX)
X_test_df = pd.DataFrame(X_test, columns=colsX)
print(X_train_df.shape)
print(X_test_df.shape)

(264, 82)
(114, 82)


In [10]:
# import numpy as np

# X_train = np.concatenate([X_train, X_test], axis=0)
# Y_train = np.concatenate([Y_train, Y_test], axis=0)

In [11]:
from sklearn.multioutput import MultiOutputRegressor
from tabpfn import TabPFNRegressor
from tabpfn.constants import ModelVersion
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Initialize the regressor
reg = MultiOutputRegressor(
    TabPFNRegressor()
)
# Uses TabPFN-2.5 weights, trained on synthetic data only.
# To use TabPFN v2:
# regressor = TabPFNRegressor.create_default_for_version(ModelVersion.V2)


In [12]:
reg.fit(X_train_df, Y_train)

c:\Users\admin\Coding\research\weld-ml\.venv\Lib\site-packages\tabpfn\validation.py:56: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  _validate_num_samples_for_cpu(


MultiOutputRegressor(estimator=TabPFNRegressor())

In [13]:
# Y_train_pred = reg.predict(X_train)
Y_test_pred = reg.predict(X_test_df)

# print(Y_train_pred.shape)
print(Y_test_pred.shape)

(114, 1)


In [14]:
def eval_perf(y_true, y_pred):
    mse = mean_squared_error(y_true=y_true, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
    r2 = r2_score(y_true=y_true, y_pred=y_pred)
    return mse, mape, r2


def print_perf(data):
    for k, v in data.items():
        print(k, ":", v)


data_arr = []
for i in range(0, _Y.shape[1]):
    mse_test, mape_test, r2_test = eval_perf(
        y_true=Y_test[:, i], y_pred=Y_test_pred[:, i]
    )

    data = {
        "Y": f"Y-{i + 1}",
        "MSE Test": mse_test,
        "MAPE Test": mape_test,
        "R2 Test": r2_test,
    }
    # print_perf(data)

    data_arr.append(data)


mse_test, mape_test, r2_test = eval_perf(y_true=Y_test, y_pred=Y_test_pred)
print_perf(data)

data = {
    "Y": "Y-All",
    "MSE Test": mse_test,
    "MAPE Test": mape_test,
    "R2 Test": r2_test,
}
data_arr.append(data)
# print_perf(data)

df_eval = pd.DataFrame.from_dict(data_arr)
display(df_eval)

if SAVE_DATA:
    filename = f"eval_{dt}.xlsx"
    df_eval.to_excel(filename, index=False)

Y : Y-1
MSE Test : 0.47019055608752536
MAPE Test : 1.314294918584433
R2 Test : 0.4487752233873018


,Y,MSE Test,MAPE Test,R2 Test
0,Y-1,0.470191,1.314295,0.448775
1,Y-All,0.470191,1.314295,0.448775


In [15]:
from tabpfn_extensions import  interpretability

shap_values = interpretability.shap.get_shap_values(
    estimator=reg,
    test_x=X_test[:4],
    attribute_names=dfX.columns.tolist(),
    algorithm="permutation",
)

PermutationExplainer explainer: 5it [04:01, 60.48s/it]                       


In [16]:
import pickle 
with open("shap_values_tabpfn.pkl", "wb") as f:
    pickle.dump(shap_values, f)